<a href="https://colab.research.google.com/github/rahul4070/CIFAR_10/blob/main/DenseNet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [4]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 70
l = 12
num_filter = 18
compression = 1
dropout_rate = 0

In [5]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

X_train =  X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# convert to one hot encoding 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170508288/170498071 [==============================] - 2s 0us/step


In [6]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
X_test.shape

(10000, 32, 32, 3)

In [8]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
    return temp

## transition Block
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [9]:
num_filter = 18
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 18)   486         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 18)  72          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 18)   0           ['batch_normalization[0][0]']

In [11]:
print(len(model.layers))

211


In [12]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

traindataGen = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,
 height_shift_range=0.1,horizontal_flip=True)

testdataGen = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,
 height_shift_range=0.1,horizontal_flip=True)

traindataGen.fit(X_train)
testdataGen.fit(X_test)

In [14]:
def lr_schedule(epoch):

    lr = 0.1
    # if epoch > 200:
    #     lr *= 0.5e-3
    # elif epoch > 140:
    #     lr *= 1e-3
    # elif epoch > 100:
    #     lr *= 1e-2
    if epoch > 200:
        lr *= 1e-2
    elif epoch > 120:
        lr *= 1e-1

    return lr

# Running the model for 70 epochs each

In [15]:
import math
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, LearningRateScheduler

lrPlateau = ReduceLROnPlateau(monitor='val_accuracy', factor=(0.1), patience=5)

model.load_weights('/content/drive/MyDrive/dense_net/DNST_model.h5')
csv_logger = CSVLogger('training.log')

callbacks_list = [lrPlateau, csv_logger]

model.fit(traindataGen.flow(X_train,y_train,batch_size=32),
                    batch_size=batch_size,
                    epochs=5,
                    steps_per_epoch=math.ceil(len(X_train) / batch_size),
                    verbose=1, 
                    validation_data=testdataGen.flow(X_test,y_test,batch_size=32),
                    callbacks = callbacks_list)

Epoch 1/5
1563/1563 [==============================] - 124s 69ms/step - loss: 0.1963 - accuracy: 0.9313 - val_loss: 0.4283 - val_accuracy: 0.8649 - lr: 0.0010
Epoch 2/5
1563/1563 [==============================] - 105s 67ms/step - loss: 0.1914 - accuracy: 0.9325 - val_loss: 0.3790 - val_accuracy: 0.8797 - lr: 0.0010
Epoch 3/5
1563/1563 [==============================] - 105s 67ms/step - loss: 0.1830 - accuracy: 0.9355 - val_loss: 0.4326 - val_accuracy: 0.8653 - lr: 0.0010
Epoch 4/5
1563/1563 [==============================] - 106s 68ms/step - loss: 0.1772 - accuracy: 0.9372 - val_loss: 0.4670 - val_accuracy: 0.8579 - lr: 0.0010
Epoch 5/5
1563/1563 [==============================] - 106s 68ms/step - loss: 0.1721 - accuracy: 0.9393 - val_loss: 0.3837 - val_accuracy: 0.8813 - lr: 0.0010


In [16]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 6s 17ms/step - loss: 0.3985 - accuracy: 0.8832
Test loss: 0.39853015542030334
Test accuracy: 0.8831999897956848


In [17]:
# Save the trained weights in to .h5 format
model.save_weights("/content/drive/MyDrive/dense_net/DNST_model.h5")
print("Saved model to disk")

Saved model to disk
